## HW1 Calibration attempt with swaption - Does work need to understand and twick the model general parameters

In [17]:
import QuantLib as ql
import numpy as Numpy

# hard-coded data
def CreateSwaptionVolatilityList():
    vol = []
    vol.append(0.11007)
    vol.append(0.11188)
    vol.append(0.1131)
    vol.append(0.1149)
    vol.append(0.11695)
    return vol

class ModelCalibrator:
    def __init__(self, endCriteria):        
        self.endCriteria = endCriteria
        self.helpers = []

    def AddCalibrationHelper(self, helper):
        self.helpers.append(helper)

    def Calibrate(self, model, engine, curve, fixedParameters):
        # assign pricing engine to all calibration helpers
        for i in range(len(self.helpers)):
            self.helpers[i].setPricingEngine(engine)
        method = ql.LevenbergMarquardt()
        if(len(fixedParameters) == 0):
            model.calibrate(self.helpers, method, self.endCriteria)
        else:
            model.calibrate(self.helpers, method, self.endCriteria,
                ql.NoConstraint(), [], fixedParameters)

# general parameters
tradeDate = ql.Date(18, ql.April, 2024)
ql.Settings.instance().evaluationDate = tradeDate
settlementDate = ql.Date(19, ql.April, 2024)
calendar = ql.TARGET()
dayCounter = ql.ActualActual(ql.ActualActual.Bond)

# create market data: term structure and diagonal volatilities
curveHandle = ql.YieldTermStructureHandle(ql.FlatForward(settlementDate, 0.04875825, ql.Actual365Fixed()))
vol = CreateSwaptionVolatilityList()

# create calibrator object
endCriteria = ql.EndCriteria(10000, 100, 0.000001, 0.00000001, 0.00000001)
calibrator = ModelCalibrator(endCriteria)

# add swaption helpers to calibrator
for i in range(len(vol)):
    t = i + 1; tenor = len(vol) - i    
    helper = ql.SwaptionHelper(
        ql.Period(t, ql.Years), 
        ql.Period(tenor, ql.Years), 
        ql.QuoteHandle(ql.SimpleQuote(vol[i])), 
        ql.USDLibor(ql.Period(3, ql.Months), curveHandle), 
        ql.Period(1, ql.Years), 
        dayCounter, 
        dayCounter, 
        curveHandle)    
    calibrator.AddCalibrationHelper(helper)

# create model and pricing engine, calibrate model and print calibrated parameters
print('case 1 : calibrate all involved parameters (HW1F : reversion, sigma)')
model = ql.HullWhite(curveHandle)
engine = ql.JamshidianSwaptionEngine(model)
fixedParameters = []
calibrator.Calibrate(model, engine, curveHandle, fixedParameters)
print('calibrated reversion: ' + str(round(model.params()[0], 5)))
print('calibrated sigma: ' + str(round(model.params()[1], 5)))
print()

print('case 2 : calibrate sigma and fix reversion to 0.05')
model = ql.HullWhite(curveHandle, 0.05, 0.0001)
engine = ql.JamshidianSwaptionEngine(model)
fixedParameters = [True, False]
calibrator.Calibrate(model, engine, curveHandle, fixedParameters)
print('fixed reversion: ' + str(round(model.params()[0], 5)))
print('calibrated sigma: ' + str(round(model.params()[1], 5)))
print()

print('case 3 : calibrate reversion and fix sigma to 0.01')
model = ql.HullWhite(curveHandle, 0.05, 0.01)
engine = ql.JamshidianSwaptionEngine(model)
fixedParameters = [False, True]
calibrator.Calibrate(model, engine, curveHandle, fixedParameters)
print('calibrated reversion: ' + str(round(model.params()[0], 5)))
print('fixed sigma: ' + str(round(model.params()[1], 5)))
print()

case 1 : calibrate all involved parameters (HW1F : reversion, sigma)
calibrated reversion: 0.07881
calibrated sigma: 0.00736

case 2 : calibrate sigma and fix reversion to 0.05
fixed reversion: 0.05
calibrated sigma: 0.00679

case 3 : calibrate reversion and fix sigma to 0.01
calibrated reversion: 0.19644
fixed sigma: 0.01



## Callable bond price calculation with HW1 - Works need to be tested (non optimized parameters)

In [2]:
import QuantLib as ql

calc_date = ql.Date(16,8,2016)
ql.Settings.instance().evaluationDate = calc_date

day_count = ql.ActualActual(ql.ActualActual.Bond)
rate = 0.035
ts = ql.FlatForward(calc_date, 
                    rate, 
                    day_count, 
                    ql.Compounded, 
                    ql.Semiannual)
ts_handle = ql.YieldTermStructureHandle(ts)

callability_schedule = ql.CallabilitySchedule()
call_price = 100.0
call_date = ql.Date(15,ql.September,2016);
null_calendar = ql.NullCalendar();
for i in range(0,24):
    callability_price  = ql.Callability(ql.BondPrice(call_price, ql.BondPrice.Clean), ql.Callability.Call, ql.Date(15,9,2016))
    callability_schedule.append(ql.Callability(ql.BondPrice(call_price, ql.BondPrice.Clean), ql.Callability.Call,call_date))

    call_date = null_calendar.advance(call_date, 3, ql.Months)
    
issue_date = ql.Date(16,ql.September,2014)        
maturity_date = ql.Date(15,ql.September,2022)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
tenor = ql.Period(ql.Quarterly)
accrual_convention = ql.Unadjusted

schedule = ql.Schedule(issue_date, maturity_date, tenor,
                       calendar, accrual_convention, accrual_convention,
                       ql.DateGeneration.Backward, False)

settlement_days = 3
face_amount = 100
accrual_daycount = ql.ActualActual(ql.ActualActual.Bond)
coupon = 0.025


bond = ql.CallableFixedRateBond(
    settlement_days, face_amount,
    schedule, [coupon], accrual_daycount,
    ql.Following, face_amount, issue_date,
    callability_schedule)

def value_bond(a, s, grid_points, bond):
    model = ql.HullWhite(ts_handle, a, s)
    engine = ql.TreeCallableFixedRateBondEngine(model, grid_points)
    bond.setPricingEngine(engine)
    return bond

value_bond(0.07881, 0.00736, 400, bond)
print("Bond price: ",bond.cleanPrice())

Bond price:  94.20656785682208


## Bond price calculation with HW1 - Doesn't work yet (returns a pointer)

In [ ]:
import QuantLib as ql

# Step 1: Date settings
calculation_date = ql.Date(1, 1, 2020)
ql.Settings.instance().evaluationDate = calculation_date

# Step 2: Bond parameters
face_value = 100
schedule = ql.Schedule(calculation_date, ql.Date(1, 1, 2030), ql.Period(ql.Semiannual),
                       ql.UnitedStates(ql.UnitedStates.NYSE), ql.Unadjusted, ql.Unadjusted,
                       ql.DateGeneration.Backward, False)
coupon_rate = 0.05
coupons = [coupon_rate]

# Step 3: Yield term structure
yield_curve = ql.FlatForward(calculation_date, 0.05, ql.Actual365Fixed())
yield_curve_handle = ql.YieldTermStructureHandle(yield_curve)

# Step 4: Define the bond
bond = ql.FixedRateBond(2, face_value, schedule, coupons, ql.Actual365Fixed())

# Step 5: Define the Hull-White model
a = 0.2
sigma = 0.10
hw_model = ql.HullWhite(yield_curve_handle, a, sigma)
engine = ql.JamshidianSwaptionEngine(hw_model)

# Step 6: Define the bond engine with the Hull-White model
bond.setPricingEngine(engine)

# Step 7: Price the bond
print("Bond price: ", bond.NPV())